# Introduction

Every semester, CMU students read through a long PDF file to figure out when their final exams are, manually aggregating the data to figure out when their final exams are.  Unlike the schedule viewer in SIO, it is really difficult to see the data with this user interface.  It would be much better if a user could view these in a calendar view, such as that provided by Google Calendar.

To solve this problem, I have written a collection of web scrapers to automatically construct a google calendar.  This notebook gives an overview of the code used for this construction and is not intended to be executable, as my code requires access to the local filesystem, which this web host does not support.  For an executable version, please visit [my github](https://github.com/jackwpenick/cmu-finals-schedule)

# Step 1:  Seamless import

I wanted my program to be as easy-to-use as possible, and figuring out what dependencies to install is one of the pain points.  I integrated the dependencies directly into the code, so that it would automatically import any missing dependencies.

In [ ]:
import sys
import os
from time import sleep
try:
    from ics import Calendar, Event
    import arrow
    from dateutil import tz
    import shutil
    import requests
    import numpy as np
    import datetime
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import PyPDF2
except ImportError:
    print("Warning: Various packages missing.  Installing...", file=sys.stderr)
    import subprocess
    subprocess.call([sys.executable] + "-m pip install ics requests numpy selenium PyPDF2".split())
    from ics import Calendar, Event
    import arrow
    from dateutil import tz
    import shutil
    import requests
    import numpy as np
    import datetime
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import PyPDF2

Anyone who has python installed, therefore, should simply be able to run my program out-of-the-box without any extra effort given to dependencies.

# Two Modalities

The user can run the program in one of two ways - with no command-line arguments, which then prompts the user to login to SIO and Google Calendar; or by passing in their AndrewId and password, which it then uses to log in to both SIO and Google Calendar (it assumes that they wish to import their calendar to the Google Calendar associated with their CMU Andrew account).

The distinction between these modalities is seamless.  For the first mode, one may just execute the program (simply double-click, if in Windows).  For the second mode, one runs the program via the command line with `python exam_schedule.py %USERNAME% %PASSWORD%`

This is seen in the main method at the end of the script:

In [ ]:
if __name__ == "__main__":
    if len(sys.argv) < 3:
        course_ids, course_friendly_names = get_courses(manual_login=True)
        exam_times = get_final_exam_times(course_ids)
        create_ics_file(course_friendly_names, exam_times, "./final_schedule.ics")
        export_google_calendar("./final_schedule.ics", manual_login=True)
    else:
        andrew_id = sys.argv[1]
        password = sys.argv[2]
        course_ids, course_friendly_names = get_courses(andrew_id, password)
        exam_times = get_final_exam_times(course_ids)
        create_ics_file(course_friendly_names, exam_times, "./final_schedule.ics")
        export_google_calendar("./final_schedule.ics", andrew_id, password)

Of course, we haven't defined the functions shown above yet, but, as you can see, the algorithm consists primarily of 4 steps.

# Step 1: Determine which courses the user is taking

This step involves scraping SIO, which is done via Selenium due to the inherent complexities of SSO.  The process at it's core is a simple idea: retrieve the SIO registration page, log in with supplied information (or wait for the user to login for us, if in the hands-on modality), and then extract the courses from the table on the page.  Since the table isn't well formatted (programmatically, at least), some basic filtering has to be done to exclude non-courses, and to distinguish between the official schedule and courses that the user is merely on the waitlist for or has on their planned course schedule. 

In [1]:
def login_with_andrew_id(driver, andrew_id, password):
    driver.find_element_by_id("j_username").send_keys(andrew_id)
    driver.find_element_by_id("j_password").send_keys(password)
    driver.find_elements_by_css_selector(".loginbutton")[0].click()

def get_courses(andrew_id="", password="", manual_login=False):
    """ Uses your andrew_id and password to retrieve the classes you are taking
        """

    course_ids = []
    course_friendly_names = {}
    
    driver = webdriver.Chrome()

    driver.get("https://s3.andrew.cmu.edu/sio/") # SSO login to here
    if manual_login:
        WebDriverWait(driver, 60 * 60).until(
            EC.title_is("CMU Student Information Online")
        )
    else:
        login_with_andrew_id(driver, andrew_id, password)
    driver.get("https://s3.andrew.cmu.edu/sio/#schedule-registration")
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "Yes"))
    )
    driver.find_element_by_id("Yes").click()
    table = driver.find_element_by_xpath(
        "//div[contains(text(),'Official Schedule')]/../../../" +
        "div[contains(@class, 'portlet-body')]/div/table/tbody"
    )
    rows = table.find_elements_by_tag_name("tr")
    for row in rows:
        cell = row.find_elements_by_tag_name("td")[0]
        if cell.text.strip() == "TITLE / NUMBER & SECTION" or \
                cell.text.strip() == "TOTAL UNITS:":
            continue
        else:
            course_ids += [(
                cell.text.strip().split()[-2],
                cell.text.strip().split()[-1],
            )]
            course_friendly_names[cell.text.strip().split()[-2]] = \
                " ".join(cell.text.strip().split()[:-2])
    return course_ids, course_friendly_names


# Step 2: Extract final exam times from the PDF

This is by far the most difficult step.  PDF files are rather complicated, and allow for many strange formatters.  The overall idea behind this is that:

1.  The extractText function in the PyPDF2 function should output something similar to the order in which the data appears.  It will at least put the exam time and date after the course number.  
2.  Dates start with a day of week
3.  Times include either an "a.m." or "p.m." designation
4.  We assume that whatever happens immidiately after both the date and the time is the location, which is a little unreasonable, but there isn't much that can be done in the way of pattern matching (e.g. some room numbers don't contain numbers, and are widely inconsistent).

This manifests itself as the following code:

In [2]:
def get_final_exam_times(course_ids):
    res = {}
    r = requests.get("https://www.cmu.edu/hub/docs/final-exams.pdf", stream=True)
    r.raw.decode_content = True
    with open("./final_schedule.pdf", "wb") as f:
        shutil.copyfileobj(r.raw, f)
    pdf = PyPDF2.PdfFileReader("./final_schedule.pdf")
    collected_lines = {}
    for i in range(pdf.getNumPages()):
        anticipate_number = None
        anticipate_section = None
        seen_section = False
        assume_next_is_room = False
        for line in pdf.getPage(i).extractText().splitlines():
            if anticipate_number is None:
                for course_number, section_id in course_ids:
                    if anticipate_number is None and course_number == line.strip():
                        anticipate_number = course_number
                        anticipate_section = section_id
            elif not seen_section:
                if anticipate_section == line:
                    seen_section = True
            elif line.strip().startswith(
                    ("Monday", "Tuesday", "Wednesday", 
                     "Thursday", "Friday", "Saturday", "Sunday")):
                collected_lines[anticipate_number, anticipate_section, "date"] = line.strip()
                if (anticipate_number, anticipate_section, "time") in collected_lines:
                    assume_next_is_room = True # no real way to make a regex or anything for this
            elif line.strip().endswith(("p.m.","a.m.")):
                collected_lines[anticipate_number, anticipate_section, "time"] = line.strip()
                if (anticipate_number, anticipate_section, "date") in collected_lines:
                    assume_next_is_room = True # no real way to make a regex or anything for this
            elif assume_next_is_room:
                collected_lines[anticipate_number, anticipate_section, "room"] = line.strip()
                assume_next_is_room = False
                anticipate_number = None
                anticipate_section = None
                seen_section = False
    absent_course_numbers = []
    for course_number, section_id in course_ids:
        if course_number in res:
            continue 
            # SIO has multiple entries, lecture number and section number.  
            # Expect exactly one of these to correspond to a final exam time entry
        try:
            date = collected_lines[course_number, section_id, "date"]
            time = collected_lines[course_number, section_id, "time"]
            room = collected_lines[course_number, section_id, "room"]
        except KeyError:
            absent_course_numbers += [course_numbers]
            continue
        first_time, second_time = time.split(" - ")[0], time.split(" - ")[1]
        first_time = first_time[:-3] + "m" # replace a.m. with am and p.
        second_time = second_time[:-3] + "m" # replace a.m. with am and p.
        res[course_number] = (
            datetime.datetime.strptime(
                    date + " " + first_time, "%A, %B %d, %Y %I:%M %p"
                ),
            datetime.datetime.strptime(
                    date + " " + second_time, "%A, %B %d, %Y %I:%M %p"
                ),
            section_id,
            room
        )
    absent_course_numbers = [
        course_number for course_number in absent_course_numbers
        if course_number not in res # ya know, multiple entries and shit
    ]
    if len(absent_course_numbers) != 0:
        print(
            "Warning: could not find final exam times for courses: %s" % 
                ",".join(absent_course_numbers),
            file=sys.stderr
        )
    return res

# Step 3: Import into an ICS file.

By far the easiest step, as there is a library to do this.

In [ ]:
def create_ics_file(course_friendly_names, exam_times, path):
    c = Calendar()
    for course_number in exam_times:
        dt_begin, dt_end, section, room = exam_times[course_number]
        e = Event(
            name="%s %s Final Exam" % (course_number, section),
            location=room,
            description="Final Exam for %s" % (course_friendly_names[course_number])
        )
        e.begin = arrow.get(dt_begin, tz.gettz('US/Eastern'))
        e.end = arrow.get(dt_end, tz.gettz('US/Eastern'))
        c.events.append(e)
    with open(path, "w") as f:
        f.writelines(c)

# Step 4: Export into Google Calendar

For this step one would think that I would use the google calendar API.  However, this requires whoever is using the program to have an API key, which would hurt usability (I would like this program to be usable by someone with 0 programming knowledge and very little computer expertise).

Therefore, I again used Selenium for this process.  This code looks a lot like the "figuring out which classes we're taking" step, though a bit more involved, as it involves a more complex login and filling out forms.

In [ ]:
def export_google_calendar(ics_filename, andrew_id="", password="", manual_login=False):
    driver = webdriver.Chrome()
    driver.get("https://calendar.google.com/")
    def patient_find_el_by_xpath_clk(path):
        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, path)))
        return driver.find_element_by_xpath(path)
    def patient_find_el_by_xpath(path):
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, path)))
        return driver.find_element_by_xpath(path)
    if manual_login:
        WebDriverWait(driver, 60 * 60).until(
            EC.url_contains("accounts.google.com")
        )
        WebDriverWait(driver, 60 * 60).until_not(
            EC.url_contains("accounts.google.com")
        )
        WebDriverWait(driver, 60 * 60).until(
            EC.url_contains("calendar.google.com")
        )
    else:
        patient_find_el_by_xpath('//input[@type="email"][@name="identifier"]') \
            .send_keys(andrew_id + "@andrew.cmu.edu")
        patient_find_el_by_xpath(
            '//div[@role="button"][./content/span[contains(text(), "Next")]]'
        ).click()
        WebDriverWait(driver, 10).until(
            EC.url_contains("login.cmu.edu")
        )
        login_with_andrew_id(driver, andrew_id, password)
    # Create Final Exam Schedule Calendar
    driver.get("https://calendar.google.com/calendar/b/1/r/settings/createcalendar")
    patient_find_el_by_xpath('//input[@type="text"][@aria-label="Name"]').send_keys("Final Exam Schedule")
    patient_find_el_by_xpath('//textarea[@aria-label="Description"]').send_keys("Auto-Generated by CMU Final Exam Schedule Google Calendar Improrter.")
    patient_find_el_by_xpath('//div[@role="button"][./content/span[contains(text(), "Create")]]').click()

    sleep(10)


    # Import it!
    driver.get("https://calendar.google.com/calendar/b/1/r/settings/export")
    patient_find_el_by_xpath('//input[@type="file"][@name="filename"]').send_keys(os.path.abspath(ics_filename))
    patient_find_el_by_xpath_clk('//div[@role="listbox"][@aria-label="Add to calendar"]').click()
    patient_find_el_by_xpath_clk('//div[@role="option"][@aria-label="Final Exam Schedule"]').click()
    patient_find_el_by_xpath_clk('//div[@role="button"][./content/span[contains(text(), "Import")]]').click()


# Conclusion

Each step in isolation is not too complicated, just some simple web-scraping and data munging.  However, in collection, the code is a useful program that allows one to view their final exam schedule in google calendar with little difficulty.